In [99]:
from selenium import webdriver
import time
import pandas as pd
import os
import re

from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

from tqdm import tqdm_notebook as tqdm
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException

import psycopg2 as ps

import spacy
from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher

### Url para vagas de analista, engenheiro e cientista de dados nas últimas 24h

In [37]:
url_ad_24 = 'https://www.linkedin.com/jobs/search?keywords=%22Analista%20De%20Dados%22&location=Brasil&locationId=&geoId=106057199&f_TPR=r86400&position=1&pageNum=0'
url_ed_24 = 'https://www.linkedin.com/jobs/search?keywords=%22Engenheiro%20De%20Dados%22&location=Brasil&locationId=&geoId=106057199&f_TPR=r86400&position=1&pageNum=0'
url_cd_go='https://www.linkedin.com/jobs/search?keywords=%22Cientista%20De%20Dados%22&location=Goi%C3%A2nia&geoId=&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0'
url_cd_24 = 'https://www.linkedin.com/jobs/search?keywords=%22Cientista%20De%20Dados%22&location=Brasil&locationId=&geoId=106057199&f_TPR=r86400&position=1&pageNum=0'

### Criando objeto do driver do chrome para abrir as urls

- Vale ressaltar que a url com a localização do driver deverá ser adaptada a localização do arquivo na sua máquina. Para fazer o download, acesse: https://chromedriver.chromium.org/downloads
- Além disso, seu Login e senha deverão ser inseridos nos campos identificados abaixo

In [45]:
driver = webdriver.Chrome(executable_path=r'C:\\Users\\t023953\\OneDrive - pti.org.br\\Documentos\\Data Science e IA\\TCC\\chromedriver_win32\\chromedriver.exe')
driver.get(url_cd_go)
driver.implicitly_wait(10)
driver.maximize_window() 
driver.switch_to.window(driver.current_window_handle)
driver.implicitly_wait(10)

### Coletando o número de vagas e realizando o scroll

In [46]:
n_vacancies= driver.find_element(By.CLASS_NAME,'results-context-header__job-count')
if n_vacancies:
    print(n_vacancies.text)
else:
    print('Classe não encontrada')

n_vacancies = n_vacancies.text.replace(".","")
n_vacancies = int(n_vacancies)

i=2
while i<=((n_vacancies/25)+1): #Procurando por todas as páginas. Ele mostra de 25 em 25
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    i+=1
    
    try:
        x=driver.find_element_by_xpath('//*[@id="main-content"]/section[2]/button')
        
        driver.execute_script("arguments[0].click();", x) #encontra o caminho e clica no botão
        time.sleep(3)
    except: #caso não tenha mais botões
        pass
        time.sleep(4)

3


### Coletandos as informações de interesse das vagas:

- Título
- Nome da empresa
- Link
- Localização 
- Tipo de emprego 
- Função
- Setor da empresa
- Nível de experiência requerido
- Descrição/texto

In [47]:
%%time

work_times = [] 
job_descs = []
fields=[]
functions=[]
experiences=[]
job_titles=[]
companies_names=[]
job_links=[]
locations=[]
job_locations=[]
job_states=[]

wait = WebDriverWait(driver, 10)
job_cards = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'base-search-card')))

for job_card in tqdm(job_cards):
    time.sleep(8)
    try:
       #Pegando a url
        job_card.click()
        job_link = driver.current_url
    except Exception:
        job_link = None
    finally:
      
        driver.back()
        wait.until(EC.visibility_of_any_elements_located((By.CLASS_NAME, 'base-search-card')))
    
    #Título da vaga
    try:
        job_title = job_card.find_element(By.CLASS_NAME, 'base-search-card__title').text
    except Exception:
        job_title = None

    #Nome da empresa
    try:
        company = job_card.find_element(By.CLASS_NAME, 'base-search-card__subtitle').text
    except Exception:
        company = None
    
    #Localização
    try:
        location_text = job_card.find_element(By.CLASS_NAME, 'job-search-card__location').text
        job_location, job_state = location_text.split(", ")
    except Exception:
        job_location = job_state = None
        
    if job_location is None:
        job_location = location_text
    
    time.sleep(3)
    # clicando no botão para exibir mais detalhes da vaga
    driver.find_element_by_xpath('/html/body/div[1]/div/section/div[2]/div/section[1]/div/div/section/button[1]').click()
    
     # Nível de experiência
    try:
        experience = driver.find_element_by_xpath('/html/body/div[1]/div/section/div[2]/div/section[1]/div/ul/li[1]/span').text
    except (NoSuchElementException, StaleElementReferenceException):
        experience = None
        

    # Tipo de emprego (tempo integral, etc)
    try:
        work_time = driver.find_element_by_xpath('/html/body/div[1]/div/section/div[2]/div/section[1]/div/ul/li[2]/span').text
    except (NoSuchElementException, StaleElementReferenceException):
        work_time = None

    # Setores
    try: 
        field = driver.find_element_by_xpath('/html/body/div[1]/div/section/div[2]/div/section[1]/div/ul/li[4]/span').text
    except (NoSuchElementException, StaleElementReferenceException):
        field = None

    # Função
    try: 
        function = driver.find_element_by_xpath('/html/body/div[1]/div/section/div[2]/div/section[1]/div/ul/li[3]/span').text
    except (NoSuchElementException, StaleElementReferenceException):
        function = None
    
    # Descrição da vaga 
    try: 
        div = driver.find_element_by_xpath('/html/body/div[1]/div/section/div[2]/div/section[1]/div/div/section/div')
        text_div = div.text
    except (NoSuchElementException, StaleElementReferenceException):
        text_div = None
    
    time.sleep(8)
    locations.append(location_text)
    job_locations.append(job_location)
    job_states.append(job_state)
    job_links.append(job_link)
    job_titles.append(job_title)
    companies_names.append(company)
    experiences.append(experience)
    work_times.append(work_time)
    fields.append(field)
    functions.append(function)
    job_descs.append(text_div)

<timed exec>:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/3 [00:00<?, ?it/s]

CPU times: total: 109 ms
Wall time: 1min 28s


### Visualizando os 5 primeiros elementos das listas geradas e seus tamanhos, para conferir se estão iguais

In [52]:
print("Dados:")
print("Localização:", locations[:5])
print("Município:", job_locations[:5])
print("Estado:", job_states[:5])
print("Link:", job_links[:5])
print("Título:", job_titles[:5])
print("Empresa:", companies_names[:5])
print("Nível de experiência:", experiences[:5])
print("Tipo de emprego:", work_times[:5])
print("Setores:", fields[:5])
print("Função:", functions[:5])
print("Descrição:", job_descs[:5])

print("Tamanhos das listas:")
print("Localização:", len(locations))
print("Município:", len(job_locations))
print("Estado:", len(job_states))
print("Link:", len(job_links))
print("Título:", len(job_titles))
print("Empresa:", len(companies_names))
print("Nível de experiência:", len(experiences))
print("Tipo de emprego:", len(work_times))
print("Setores:", len(fields))
print("Função:", len(functions))
print("Descrição:", len(job_descs))

Dados:
Localização: ['Goiânia, GO', 'Goiânia, GO', 'Goiânia, GO']
Município: ['Goiânia', 'Goiânia', 'Goiânia']
Estado: ['GO', 'GO', 'GO']
Link: ['https://www.linkedin.com/jobs/search?keywords=%22Cientista%20De%20Dados%22&location=Goi%C3%A2nia&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0&currentJobId=3612929000', 'https://www.linkedin.com/jobs/search?keywords=%22Cientista%20De%20Dados%22&location=Goi%C3%A2nia&trk=public_jobs_jobs-search-bar_search-submit&position=2&pageNum=0&currentJobId=3612718687', 'https://www.linkedin.com/jobs/search?keywords=%22Cientista%20De%20Dados%22&location=Goi%C3%A2nia&trk=public_jobs_jobs-search-bar_search-submit&position=3&pageNum=0&currentJobId=3628742869']
Título: ['Pessoa Cientista de Dados Sênior', 'Cientista de Dados Sênior', 'Cientista de Dados Sênior']
Empresa: ['Minsait', 'Minsait', 'Empresa Confidencial']
Nível de experiência: ['Não aplicável', 'Não aplicável', 'Tempo integral']
Tipo de emprego: ['Tempo integral', 'Tempo integ

### Isolando os ids das vagas a partir de seus links:

- O objetivo é verificar se as vagas já não existem no banco de dados, para evitar registros duplicados

In [53]:
from urllib.parse import urlparse, parse_qs
def isolate_link_ids(job_links):

    job_ids = []

    for link in job_links:
        url = urlparse(link)
        parameters = parse_qs(url.query)
        job_id = parameters.get('currentJobId', [''][0])
        job_ids.extend(job_id)
    
    return job_ids

job_ids = isolate_link_ids(job_links)
print(len(job_ids))
print(job_ids)

3
['3612929000', '3612718687', '3628742869']


### Criando data_frame para eliminar os registros de vagas que não contenham os títulos de "Analista de Dados", "Cientista de Dados" e "Engenheiro de Dados". 

- Tal processo foi necessário, visto que em alguns casos - como Engenheiro de Dados - por mais que a string de busca fosse essa, resultados como "Engenheiro Civil" e "Engenheiro Eletricista" eram retornados

In [58]:
import pandas as pd

df = pd.DataFrame({
    'locations': locations,
    'job_locations': job_locations,
    'job_states': job_states,
    'job_links': job_links,
    'job_titles': job_titles,
    'companies_names': companies_names,
    'experiences': experiences,
    'work_times': work_times,
    'fields': fields,
    'functions': functions,
    'job_descriptions': job_descs,
    'job_ids': job_ids
})

#Eliminando vagas na qual o título não contem os textos "Analista, Cientista ou engenheiro de dados, assim como suas versões em inglês"
job_titles_to_maintain = '(analista|cientista|engenheiro)\s*de\s*dados|data\s*(analyst|scientist|engineer)'
df_filtered = df[df['job_titles'].str.contains(job_titles_to_maintain, case=False)]
print(df_filtered)

     locations job_locations job_states  \
0  Goiânia, GO       Goiânia         GO   
1  Goiânia, GO       Goiânia         GO   
2  Goiânia, GO       Goiânia         GO   

                                           job_links  \
0  https://www.linkedin.com/jobs/search?keywords=...   
1  https://www.linkedin.com/jobs/search?keywords=...   
2  https://www.linkedin.com/jobs/search?keywords=...   

                         job_titles       companies_names     experiences  \
0  Pessoa Cientista de Dados Sênior               Minsait   Não aplicável   
1         Cientista de Dados Sênior               Minsait   Não aplicável   
2         Cientista de Dados Sênior  Empresa Confidencial  Tempo integral   

       work_times                        fields  \
0  Tempo integral  Serviços e consultoria de TI   
1  Tempo integral  Serviços e consultoria de TI   
2            None                          None   

                               functions  \
0  Engenharia e Tecnologia da informação   


C:\Users\t023953\AppData\Local\Temp\ipykernel_2976\698470714.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_filtered = df[df['job_titles'].str.contains(job_titles_to_maintain, case=False)]


### Verificando se as datas já não existem no banco pelo id da vaga. Caso esteja, remove do dataframe

- Para esse trabalho, utilizou-se o gerenciador de banco de dados PostgreSQL. Porém, o código é adaptável. 

In [59]:
import psycopg2 as ps

# create connection
connection = ps.connect(
    host='localhost',
    database='database', #entrar com os dados da sua database
    user='user', #entrar com seu user
    password='senha' #entrar com sua senha
)

existing_job_links = []

with connection:
    with connection.cursor() as cursor:
        query = "SELECT link_id FROM jobs WHERE link_id IN %s"
        cursor.execute(query, (tuple(job_ids),))
        rows = cursor.fetchall()
        for row in rows:
            existing_job_links.append(row[0])
            
print(existing_job_links)

df_filtered = df_filtered[~df_filtered['job_ids'].isin(existing_job_links)]
print(df_filtered)


['3612929000', '3612718687']
     locations job_locations job_states  \
2  Goiânia, GO       Goiânia         GO   

                                           job_links  \
2  https://www.linkedin.com/jobs/search?keywords=...   

                  job_titles       companies_names     experiences work_times  \
2  Cientista de Dados Sênior  Empresa Confidencial  Tempo integral       None   

  fields functions                                   job_descriptions  \
2   None      None  Contratação: CLT\nFormato: Presencial\nLocalid...   

      job_ids  
2  3628742869  


### Inserindo as vagas no banco de dados

- Nesse etapa, foi setada a coluna "categoria" de acordo com os títulos da vaga
- Além disso, tendo em vista que a classificação do LinkedIn para o nível de senioridade das vagas não é tão precisa, resolvi extrair dos títulos das próprias vagas

In [66]:
list_of_seniority=['Estágio', 'Estagiário', 'Estagiario', 'Júnior', 'Jr','Estagio', 'Pleno a Sênior', 'Junior', 'Pleno', 'PL', 'Sênior', 'Senior', 'Sr', 'I', 'II', 'III', '1', '2', '3']
def map_experience_category(level):
    if level.lower() in ['estágio', 'estagiário', 'estagio']:
        return 'Estagiário'
    elif level.lower() in ['júnior', 'jr', 'i', '1', 'junior']:
        return 'Júnior'
    elif level.lower() in ['pleno', 'pl', 'ii', '2']:
        return 'Pleno'
    elif level.lower() in ['sênior', 'senior', 'sr', 'iii', '3']:
        return 'Sênior'
    else:
        return level
    
def find_seniority(title):
    matches = [seniority for seniority in list_of_seniority if re.search(fr"\b{seniority}\b", title, flags=re.IGNORECASE)]
    return matches[0] if matches else "Não Informado"

In [68]:
import psycopg2 as ps

connection = ps.connect(
    host='localhost',
    database='database', #sua database
    user='user', #seu usuário
    password='senha' #sua senha
)

cursor = connection.cursor()
job_ids = isolate_link_ids(job_links)

#encontra o nível de senioridade
df_filtered['level_of_experience']=None
df_filtered['level_of_experience']=df_filtered['job_titles'].apply(find_seniority)

for index, row in df_filtered.iterrows():
    job_title = row['job_titles']
    category = ''

    # Setar a categoria
    if 'cientista de dados' in job_title.lower() or 'cientista dados' in job_title.lower() or 'data scientist' in job_title.lower():
        category = 'Cientista de Dados'
    elif 'engenheiro de dados' in job_title.lower() or 'engenheiro dados' in job_title.lower() or 'data engineer' in job_title.lower():
        category = 'Engenheiro de Dados'
    elif 'analista de dados' in job_title.lower() or 'analista dados' in job_title.lower() or 'data analyst' in job_title.lower():
        category = 'Analista de Dados'

    # Executar a inserção no banco de dados
    cursor.execute(
        """
        INSERT INTO jobs_2_teste (job_title, level_of_experience, job_link, job_desc, job_location, job_state, company_name, experience, work_time, field, function, link_id, category)
        VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """,
        (row['job_titles'], row['level_of_experience'], row['job_links'], row['job_descriptions'], row['job_locations'], row['job_states'], row['companies_names'], row['experiences'], row['work_times'], row['fields'], row['functions'], job_ids[index], category)
    )

connection.commit()
cursor.close()
connection.close()

### Extração das modalidades de trabalho:

- Para a extração das modalidades de trabalho, foi necessário fazer Login. Os demais dados foram extraídos sem a realização do login, visto que a localização XPATH dos campos sofria menos alteração se nenhum usuário tivesse sido informado, já que com login, mais dados apareciam e consequentemente a estrutura da página variava com mais frequência. Contudo, para a extração da informação de modalidade de trabalho, tal trabalho foi necessário

In [98]:
driver = webdriver.Chrome(executable_path=r'C:\\Users\\t023953\\OneDrive - pti.org.br\\Documentos\\Data Science e IA\\TCC\\chromedriver_win32\\chromedriver.exe')
driver.get('https://br.linkedin.com/?original_referer=')
driver.refresh()
driver.find_element_by_xpath('//*[@id="session_key"]').send_keys("email") #insira aqui o seu e-mail
driver.find_element_by_xpath('//*[@id="session_password"]').send_keys("senha") #insira aqui sua senha
time.sleep(1)

driver.find_element_by_xpath('//*[@id="main-content"]/section[1]/div/div/form/div[2]/button').click()

df_filtered['work_models'] = None
i = 0
for index, row in tqdm(df_filtered.iterrows(), total=len(df_filtered)):
    id = row['job_ids']
    link = row['job_links']
    model = row['work_models']

    try:
        driver.get(link)
        time.sleep(4)
        try:
            model = driver.find_element_by_xpath('//*[@id="main"]/div/div[2]/div/div[2]/div[1]/div/div[1]/div/div[1]/div[1]/div[2]/span[1]/span[3]').text
        except:
            model = None
        
        df_filtered.at[index, 'work_models'] = model  

    except:
        pass
    
    time.sleep(2)
    i += 1
    

#atualizando os dados no banco
connection = ps.connect(
    host='localhost',
    database='tcc_pucrs',
    user='postgres',
    password='postgres'
)

cursor = connection.cursor()

for index, row in df_filtered.iterrows():
    work_model = row['work_models']
    job_id = row['job_ids']
    query = f"UPDATE jobs SET work_model = '{work_model}' WHERE link_id = '{str(job_id)}';"
    cursor.execute(query)

# Confirmar as alterações e fechar a conexão
connection.commit()
cursor.close()
connection.close()

C:\Users\t023953\AppData\Local\Temp\ipykernel_2976\3867213903.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for index, row in tqdm(df_filtered.iterrows(), total=len(df_filtered)):


  0%|          | 0/1 [00:00<?, ?it/s]

# --------------------------------SEÇÃO 2: ANÁLISE DE KEY-WORDS--------------------------

### Importação dos dados do banco de dados:

- Importando anúncios de emprego que ainda não foram analisados
- Importando os benefícios mapeados e as skills/competências/tecnologias

In [117]:
connection = ps.connect(
    host='localhost',
    database='tcc_pucrs',
    user='postgres',
    password='postgres'
)

cursor = connection.cursor()

#apenas vagas que ainda não estão no banco de key words/não foram analisadas
query = "Select j.* from jobs j left join jobs_skills js on j.id=js.jobs_id where js.jobs_id is null;"
cursor.execute(query)
result = cursor.fetchall()
fields = [columns[0] for columns in cursor.description]
df = pd.DataFrame(result, columns = fields)

#Consulta a lista de skills da tabela key_skills
query_2 = "Select description from key_skills;"
cursor.execute(query_2)
list_of_skills = cursor.fetchall()
list_of_skills = [skill[0] for skill in list_of_skills]

#Consulta os modelos de trabalho
query_3 = "Select description from work_models;"
cursor.execute(query_3)
work_models = cursor.fetchall()
work_models = [model[0] for model in work_models]

#Consulta os benefícios
query_4 = "Select description from benefits;"
cursor.execute(query_4)
benefits = cursor.fetchall()
benefits = [b[0] for b in benefits]

print(df)

cursor.close()
connection.close()

list_of_skills.extend(work_models)
list_of_skills.extend(benefits)
list_of_skills = [skill.lower() for skill in list_of_skills]

   id                  job_title  \
0   6  Cientista de Dados Sênior   

                                                                                                                                                                                      job_link  \
0  https://www.linkedin.com/jobs/search?keywords=%22Cientista%20De%20Dados%22&location=Goi%C3%A2nia&trk=public_jobs_jobs-search-bar_search-submit&position=3&pageNum=0&currentJobId=3628742869   

  job_location job_state          company_name publication_date  \
0      Goiânia        GO  Empresa Confidencial             None   

       experience work_time field function            category     link_id  \
0  Tempo integral      None  None     None  Cientista de Dados  3628742869   

                                                                                                                                                                                                                                                      

## Função para remover pontuações (.,;-) das descrições de emprego
- Estava dando erro na identificação de algumas palavras-chaves das vagas porque estava com ponto final (Ex: R.). Além disso, substituir a / por um espaço em branco

In [118]:
def clean_text(text):
    
    regex_expression = r'[.,;/-]' 
    cleaned_text = re.sub(regex_expression, ' ', text).lower()
    return cleaned_text

### Função para extração de palavras-chave

In [121]:
nlp = spacy.load("pt_core_news_sm")

dictionary_of_synonyms = {
    "power bi": ["powerbi", "power-bi", "microsoft power bi"], #
    "python": ["py", "python3", "python2", "py3", "py2"],
    "aws": ["amazon web services", "aws cloud"],
    "jupyter notebook": ["jupyter"],
    "agile": ["metodologias ágeis", "metodologias ageis", "agile methodologies", "metodologia ágil", "metodologia agil"],
    "excel": ["microsoft excel"],
    "microsoft sql server": ["sql server", "ms sql server", "sql server da microsoft"],
    "árvores de decisão": ["árvore de decisão", "decision tree", "decision trees"],
    "amazon redshift": ["redshift"],
    "apache spark": ["spark"],
    "deep learning": ["aprendizado profundo"],
    "machine learning": ["aprendizado de máquina"],
    "tensorflow": ["tensor flow"],
    "data studio": ["google data studio"],
    "bigquery": ["big query"],
    "estatística": ["estatísticas", "statistics"],
    "scikit-learn": ["scikit learn"],
    "banco de dados": ["bancos de dados"],
    "presencial": ["presencialmente"],
    "home office": ["home-office"],
    "remoto": ["remota", "remotos", "remotas"],
    "híbrida": ["híbrido"],
    "vale alimentação": ["alimentação"],
    "vale refeição": ["refeição"],
    "vale transporte": ["transporte"],
    "auxílio pet": [],
    "previdência": [],
    "auxílio babá": [],
    "licença parental": ["licença paterna"],
    "desconto": [],
    "assistência médica": ["plano de saúde"],
    "gym pass": ["gympass"],
    "décimo quarto": ["14°"],
    "day off": ["dayoff"],
    "odontológica": ["odontológico", "assistência odontológica"],
    "seguro de vida": [],
    "psicológico": [],
    "auxílio creche": [],
    "pentaho": ["pentaho data integration"],
    "ppr": ["participação nos resultados"],
    "plr": ["pl", "participação nos lucros"]
}


reverse_synonyms = {syn: original for original, synonyms in dictionary_of_synonyms.items() for syn in synonyms}
# Inicializando o PhraseMatcher
phrase_matcher = PhraseMatcher(nlp.vocab, attr='LOWER')

# Transforma cada skill em um documento do spaCy e adiciona ao PhraseMatcher
phrase_patterns = [nlp.make_doc(skill) for skill in list_of_skills]
phrase_matcher.add('SKILL', None, *phrase_patterns)


# Corrige a função extract_skills para considerar sinônimos e preservar a forma original da habilidade
def extract_skills(job_desc, list_of_skills):
    extracted_skills = set()
    document = nlp(job_desc) #processa o texto da descrição para criar o documento
    
    # Correspondências de frase
    matches = phrase_matcher(document) #encontra os matchs/correspondências
    
    for match_id, start, end in matches: #para cada correspondência, extrai e verifica se é um sinônimo
        span = document[start:end] #pega a skill. start e end do match do documento com a descrição inteira da vaga
        skill = span.text
        # Se a habilidade é um sinônimo, adicione a forma original da habilidade
        if skill.lower() in reverse_synonyms:
            extracted_skills.add(reverse_synonyms[skill.lower()])
        else:
            extracted_skills.add(skill)

    return list(extracted_skills)

### Chamando a função de extração das skills/key words/benefícios para cada vaga

In [122]:
df_skills = pd.DataFrame(columns=['id', 'extracted_skills'])

def call_function_extract_skills(row):
    job_id = row["id"]
    job_desc = row["job_desc"]
    extracted_skills = extract_skills(clean_text(job_desc), list_of_skills)
    return pd.Series({"id": job_id, "extracted_skills": extracted_skills})

df_skills = df.apply(call_function_extract_skills, axis=1)
print(df_skills)

   id  \
0   6   

                                                                                                                                                                                                                                                                                                                                                                                              extracted_skills  
0  [matplotlib, r, hadoop, banco de dados, mongodb, mysql, estatística, odontológica, seguro de vida, desconto, mineração de dados, nosql, visualização de dados, postgresql, gym pass, plotly, assistência médica, apache spark, big data, tableau, power bi, vale transporte, sql, python, vale refeição, git, vale alimentação, presencial, gitlab, machine learning, cassandra, ggplot2, jupyter notebook]  


### Inserindo os dados na tabela job_skills

In [ ]:
connection = ps.connect(
    host='localhost',
    database='database', #sua database
    user='user',#seu user
    password='senha' #sua senha
)

cursor = connection.cursor()

for _, row in df_skills.iterrows(): #Interage a cada linha
    job_id = row['id']
    extracted_skills = row['extracted_skills']
    
    for skill in extracted_skills:
        if skill in [b.lower() for b in benefits]:
            cursor.execute("SELECT id FROM benefits WHERE lower(description)=trim(%s)", (skill.lower(),))
            benefit_id = cursor.fetchone()[0]
            query = f"INSERT INTO jobs_benefits (jobs_id, benefits_id) VALUES ({job_id}, '{benefit_id}')"
        else:
            cursor.execute("SELECT id FROM key_skills WHERE lower(description)=trim(%s)", (skill.lower(),))
            skill_id = cursor.fetchone()[0]
            query = f"INSERT INTO jobs_skills (jobs_id, key_skills_id) VALUES ({job_id}, '{skill_id}')"
            
        cursor.execute(query)

connection.commit()
cursor.close()
connection.close()